In [ ]:
!pip install langchain
!pip install openai

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = "발급받은 API 키"

## 자동으로 퍼즐을 푸는 간단한 예

In [ ]:
!pip install Wikipedia

In [ ]:
from langchain.tools import WikipediaQueryRun
from langchain.agents import initialize_agent
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools import PythonREPLTool
from langchain.chat_models import ChatOpenAI

python_repl = PythonREPLTool()
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

llm = ChatOpenAI(temperature=0.5)
tools = [python_repl, wikipedia]

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True, handle_parsing_errors=True)

In [ ]:
agent.run(
    """
    먼저 스도쿠의 규칙을 확인하고, 그 규칙에 따라 다음 스도쿠 퍼즐을 풀어주세요.
    출력은 그대로 배열로 해주셔도 됩니다.:
    [
        [4, 0, 9, 1, 8, 0, 0, 0, 0],
        [1, 7, 0, 5, 0, 0, 0, 3, 0],
        [0, 3, 0, 0, 0, 0, 0, 0, 8],
        [0, 0, 0, 4, 0, 2, 0, 0, 0],
        [0, 9, 0, 0, 0, 0, 2, 4, 0],
        [2, 0, 3, 0, 0, 0, 0, 0, 5],
        [0, 1, 0, 0, 0, 0, 0, 0, 3],
        [3, 0, 7, 0, 0, 1, 8, 0, 9],
        [0, 0, 0, 0, 9, 7, 1, 6, 0],
    ]
    """
)

## 웹 검색에 대응하기

In [ ]:
import os

os.environ["GOOGLE_CSE_ID"] = "Google PSE ID"
os.environ["GOOGLE_API_KEY"] = "Google API Key"

In [ ]:
!pip install google-search

In [ ]:
from langchain.tools import Tool
from langchain.utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()
tool = Tool(
    name = "Google Search",
    description="Search Google for recent results.",
    func=search.run
)

In [ ]:
result = tool.run("올해 G7 개최국")
print(result)

In [ ]:
from langchain.agents import initialize_agent
from langchain_experimental.tools import PythonREPLTool
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool
from langchain.utilities import GoogleSearchAPIWrapper

llm = ChatOpenAI(temperature=0.2)
python_repl = PythonREPLTool()

tools = [python_repl]

search = GoogleSearchAPIWrapper()

tools.append(
    Tool(
        name = "Google Search",
        description="Search Google for recent results.",
        func=search.run
    )
)

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [ ]:
agent.run("내일 서울의 날씨 예보는?")

## 색인 데이터베이스 검색 통합하기

In [ ]:
!pip install llama-index

In [ ]:
from langchain.chat_models import ChatOpenAI
from llama_index.core import VectorStoreIndex, download_loader
from langchain.agents import initialize_agent, Tool

BeautifulSoupWebReader = download_loader("BeautifulSoupWebReader")

documents = BeautifulSoupWebReader().load_data(urls=
    [
        'https://www.gutenberg.org/cache/epub/59/pg59-images.html'
    ]
)
index = VectorStoreIndex.from_documents(documents=documents)
tools = [
    Tool(
        name="LlamaIndex",
        func=lambda q: str(index.as_query_engine().query(q)),
        description="데카르트의 방법서론에 대한 질문에 답할 때 편리합니다."
    ),
]

llm = ChatOpenAI(temperature=0)

agent = initialize_agent(tools=tools, llm=llm, agent="zero-shot-react-description", verbose=True)

In [ ]:
agent.run("데카르트의 방법서설에서 과학적 방법에 대해 어떻게 설명하고 있나요?")

## PDF 문서 검색하기

In [ ]:
from pathlib import Path
from llama_index.core import download_loader

CJKPDFReader = download_loader("CJKPDFReader")

loader = CJKPDFReader()
documents = loader.load_data(file=Path('./civilcode.pdf'))

In [ ]:
from llama_index.core import VectorStoreIndex

# 색인 생성
index = VectorStoreIndex.from_documents(documents)

In [ ]:
# 색인 저장
index.storage_context.persist('index_civilcode')

In [ ]:
from llama_index.core import StorageContext, load_index_from_storage

# 색인 적재
storage_context = StorageContext.from_defaults(persist_dir="./index_civilcode")
index = load_index_from_storage(storage_context)
query_engine = index.as_query_engine()

In [ ]:
print(query_engine.query("유언에 관한 법률은 몇 조에 있고 내용은 무엇인지 한국어로 답해주세요.)"))

## 여러 개의 색인 데이터베이스를 각각 별도의 도구로 생성하기

In [ ]:
documents = loader.load_data(file=Path('./criminalcode.pdf'))
index_criminalcode = VectorStoreIndex.from_documents(documents)
index_criminalcode.storage_context.persist('index_criminalcode')

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from llama_index.core import StorageContext, load_index_from_storage

storage_context = StorageContext.from_defaults(persist_dir="./index_civilcode")
index_civilcode = load_index_from_storage(storage_context)

storage_context = StorageContext.from_defaults(persist_dir="./index_criminalcode")
index_criminalcode = load_index_from_storage(storage_context)

llm = ChatOpenAI(temperature=0)

tools = [
    Tool(
        name="civilcode",
        func=lambda q: str(index_civilcode.as_query_engine().query(q)),
        description="민법에 관련된 질문에 답할 때 편리합니다.",
        return_direct=True,
    ),
    Tool(
        name="criminalcode",
        func=lambda q: str(index_criminalcode.as_query_engine().query(q)),
        description="형법에 관련된 질문에 답할 때 편리합니다.",
        return_direct=True,
    ),
]

agent = initialize_agent(tools=tools, llm=llm, agent="zero-shot-react-description", verbose=True)

In [ ]:
agent.run("상속의 승인과 포기에 대해 한국어로 알려주세요.")

In [ ]:
agent.run("외교상의 기밀 누설에 관한 법률과 처벌 내용을 한국어로 알려주세요.")

In [ ]:
agent = initialize_agent(tools=tools, llm=llm, agent="zero-shot-react-description", verbose=True, handle_parsing_errors=True)
agent.run("안녕하세요")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from llama_index.core import StorageContext, load_index_from_storage
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

storage_context = StorageContext.from_defaults(persist_dir="./index_civilcode")
index_civilcode = load_index_from_storage(storage_context)

storage_context = StorageContext.from_defaults(persist_dir="./index_criminalcode")
index_criminalcode = load_index_from_storage(storage_context)

# 범용적으로 일반적인 대화를 하기 위해 단일 체인을 생성해 도구로 등록
prompt = PromptTemplate(input_variables=["query"], template="{query}")
chain_general = LLMChain(llm=ChatOpenAI(), prompt=prompt)

llm = ChatOpenAI(temperature=0)

tools = [
    Tool(
        name="civilcode",
        func=lambda q: str(index_civilcode.as_query_engine().query(q)),
        description="민법에 관련된 질문에 답할 때 편리합니다.",
        return_direct=True,
    ),
    Tool(
        name="criminalcode",
        func=lambda q: str(index_criminalcode.as_query_engine().query(q)),
        description="형법에 관련된 질문에 답할 때 편리합니다.",
        return_direct=True,
    ),
    Tool(
        name="general",
        func=chain_general.run,
        description="범용적인 질문에 답할 때 편리합니다.",
        return_direct=True,
    ),
]

agent = initialize_agent(tools=tools, llm=llm, agent="zero-shot-react-description", verbose=True, handle_parsing_errors=True)

In [ ]:
agent.run("안녕하세요")

## SQL 데이터베이스 검색 통합하기

In [ ]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

sql_uri = "sqlite:///user_support.db"
db = SQLDatabase.from_uri(sql_uri)
llm = ChatOpenAI(temperature=0)

template="""
    {query}
    """

prompt = PromptTemplate(input_variables=["query"], template=template)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

tools = [
    Tool(
        name="SQL Database",
        func=db_chain.run,
        description="데이터베이스 질의에 답할 때 편리합니다.",
        return_direct=True
    ),
]

agent = initialize_agent(tools=tools, llm=llm, agent="zero-shot-react-description", verbose=True, handle_parsing_errors=True)

In [ ]:
agent.run("윤 태진님의 사용자 ID를 알려주세요. 한국어는 성, 이름 순서로 이름을 씁니다.")

In [ ]:
agent.run("사용자 ID=1의 구매 이력을 알려주세요.")